In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from numba import njit, prange
import otbApplication
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage

# Guidelines
This small notebook aims to convert a raster (Ground Truth in my case) so that it can be compared cell by cell to another raster.

The final rasters will have the same coordinate system, raster size and cell size.

What we do in this notebook:
* Crop the DSM
* Load the rasters and their metadata
* Reproject them in the same coordinate system
* Resample the nodata mask to the desired size and filter it (because some artefacts appear on NaN)
* Fill the nodata holes from the raster and resample it to the desized size
* Apply the resampled nodata mask to the resampled raster

Voilà !

# Extracting the ROI

In the following example, the ground truth DSM is larger than the CARS DSM

In [ ]:
dsm_gt = "/work/CAMPUS/etudes/3D/Development/malinoro/Glacier/DEM/peyto_20160913_1x1_medfilt-median-DEM.tif"
dsm_gt_cropped = "/work/CAMPUS/etudes/3D/Development/malinoro/CARS_output/Peyto_ROI/peyto_20160913_1x1_medfilt-median-DEM.tif"

reference_raster = "/work/CAMPUS/users/cuervog/git/cars/images/Peyto/Peyto_ROI_Gab/Peyto_ROI/extract_dsm.tif"

In [ ]:
app = otbApplication.Registry.CreateApplication("ExtractROI")

app.SetParameterString("in", dsm_gt)
app.SetParameterString("mode","fit")
app.SetParameterString("mode.fit.im", reference_raster)
app.SetParameterString("out", dsm_gt_cropped)

In [ ]:
app.ExecuteAndWriteOutput()

# Reading the Data and Metadata

In [ ]:
with rasterio.open(reference_raster, "r") as f:
    dsm_profile = f.profile
    dsm_bounds = f.bounds
    print(dsm_profile)
    dsm_array = f.read(1)
    dsm_array[dsm_array==dsm_profile["nodata"]] = np.nan
    

with rasterio.open(dsm_gt_cropped) as f:
    gt_profile = f.profile
    gt_bounds = f.bounds
    gt_meta = f.meta
    print(gt_profile)
    gt = f.read(1)

# Reprojecting the GT

In [ ]:
transform, width, height = calculate_default_transform(
    gt_profile["crs"], dsm_profile["crs"], gt_profile["width"], gt_profile["height"], *gt_bounds)
kwargs = gt_meta.copy()
kwargs.update({
    'crs': dsm_profile["crs"],
    'transform': transform,
    'width': width,
    'height': height
})

gt_reprojected, gt_transform = reproject(source=gt,
                                       src_transform=gt_profile["transform"],
                                       src_crs=gt_profile["crs"],
                                       dst_crs=dsm_profile["crs"],
                                       resampling=Resampling.nearest)
gt_reprojected = gt_reprojected[0]
gt_reprojected[gt_reprojected==gt_profile["nodata"]] = np.nan
gt[gt==gt_profile["nodata"]] = np.nan

In [ ]:
np.unique(gt_reprojected)

# Computing the pixel size ratios

In [ ]:
ratio_x, ratio_y = dsm_profile["height"] / gt_profile["height"], dsm_profile["width"] / gt_profile["width"]
ratio_x, ratio_y

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
ax.imshow(dsm_array)
ax.grid(False)
ax.set_title("CARS DSM")

ax = fig.add_subplot(122)
ax.imshow(gt_reprojected)
ax.set_title("Reprojected GT")
ax.grid(False)

# Creating and projecting a NaN mask for the final result

In [ ]:
"""TODO adapter la taille du kernel au ratio x,y"""
gt_mask = np.isnan(gt_reprojected)
result_mask = ndimage.zoom(gt_mask, (ratio_x, ratio_y), order=0)

#kernel_size = (5, 5)
kernel_size = (1,1)
result_mask_view = sliding_window_view(np.pad(result_mask, pad_width=[(kernel_size[0]//2, kernel_size[0]//2), (kernel_size[1]//2, kernel_size[1]//2)], mode="edge"), kernel_size)
filtered_result = np.any(result_mask_view, axis=(2,3))

In [ ]:
print("SRC shape:", result_mask.shape)
print("GT shape:", gt_mask.shape)

In [ ]:
print("Adjust the zoom figure to see if the kernel size is correct")
left_src, right_src, bottom_src, top_src = 3400, 3600, 4600, 4400
left_gt, right_gt, bottom_gt, top_gt  = 1700, 1800, 2300, 2200

fig = plt.figure(figsize=(20, 14))

ax = fig.add_subplot(231)
im_1 = np.zeros(gt_mask.shape+(4,), dtype=int)
im_1[gt_mask] = (255, 0, 0, 255)
ax.imshow(im_1)
ax.set_title("No Data GT")
ax.grid(False)

ax = fig.add_subplot(232)
im_2 = np.zeros(result_mask.shape+(4,), dtype=int)
im_2[result_mask] = (255, 0, 0, 255)
ax.imshow(im_2)
ax.set_title("No Data after resampling")
ax.grid(False)

ax = fig.add_subplot(233)
im_3 = np.zeros(filtered_result.shape+(4,), dtype=int)
im_3[filtered_result] = (255, 0, 0, 255)
ax.imshow(im_3)
ax.set_title("No Data after sampling, filtered")
ax.grid(False)

ax = fig.add_subplot(234)
im_1 = np.zeros(gt_mask.shape+(4,), dtype=int)
im_1[gt_mask] = (255, 0, 0, 255)
ax.imshow(im_1[top_gt:bottom_gt+1, left_gt:right_gt+1], extent=(left_gt, right_gt, bottom_gt, top_gt))
ax.set_title("No Data GT")
ax.grid(False)

ax = fig.add_subplot(235)
im_2 = np.zeros(result_mask.shape+(4,), dtype=int)
im_2[result_mask] = (255, 0, 0, 255)
ax.imshow(im_2[top_src:bottom_src+1, left_src:right_src+1], extent=(left_src, right_src, bottom_src, top_src))
ax.set_title("No Data after resampling")
ax.grid(False)

ax = fig.add_subplot(236)
im_3 = np.zeros(filtered_result.shape+(4,), dtype=int)
im_3[filtered_result] = (255, 0, 0, 255)
ax.imshow(im_3[top_src:bottom_src+1, left_src:right_src+1], extent=(left_src, right_src, bottom_src, top_src))
ax.set_title("No Data after sampling, filtered")
ax.grid(False)

# Filling the holes before resampling

In [ ]:
@njit("f4[:,:](f4[:,:])", parallel=True)
def expand_non_nans(array):
    n_rows, n_cols = array.shape
    array_out = np.copy(array)
    for j in prange(n_cols):
        if np.isnan(array[0, j]):
            array_out[0, j] = array_out[1, j] 
        if np.isnan(array[n_rows - 1, j]):
            array_out[n_rows - 1, j] = array_out[n_rows - 2, j]
    for i in prange(n_rows):
        if np.isnan(array[i, 0]):
            array_out[i, 0] = array_out[i, 1]
        if np.isnan(array[i, n_cols-1]):
            array_out[i, n_cols-1] = array_out[i, n_cols-2]
    
    for i in prange(1, n_rows - 1):
        for j in prange(1, n_cols - 1):
            if np.isnan(array[i, j]):
                array_out[i, j] = np.nanmean(np.array([array[i, j-1], array[i, j+1], array[i-1, j], array[i+1, j]]))
    return array_out

In [ ]:
# If there are NaN holes to big, this takes forever.
# So we limit the number of iterations
gt_copy = np.copy(gt_reprojected)

iterations = 1
while iterations <= 10:
    print(f"\rBegining iter {iterations}", end="")
    if ~(np.isnan(gt_copy).any()):
        print("\nNo more NaNs!")
        break
    else:
        gt_copy = expand_non_nans(gt_copy)
    iterations += 1

if iterations == 11:
    print("\nForcing rest of NaNs to 0")
    gt_copy[np.isnan(gt_copy)] = 0

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
ax.imshow(gt_reprojected)
ax.set_title("GT reprojected")
ax.grid(False)

ax = fig.add_subplot(122)
ax.imshow(gt_copy)
ax.set_title("GT reprojected with filled holes")
ax.grid(False)

# Resampling the reprojected images and masking NaN

In [ ]:
gt_result = ndimage.zoom(gt_copy, (ratio_x, ratio_y), order=0)
gt_result[filtered_result] = np.nan

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(131)
ax.imshow(dsm_array)
ax.set_title(f"CARS DSM ({dsm_array.shape[0]} x {dsm_array.shape[1]})")
ax.grid(False)

ax = fig.add_subplot(132)
ax.imshow(gt)
ax.set_title(f"GT ({gt.shape[0]} x {gt.shape[1]})")
ax.grid(False)

ax = fig.add_subplot(133)
ax.imshow(gt_result)
ax.set_title(f"GT reprojected, resampled  ({gt_result.shape[0]} x {gt_result.shape[1]})")
ax.grid(False)

# Saving the data

In [ ]:
output_path_name = os.path.join(os.path.dirname(reference_raster), "gt_resampled.tif")
output_path_name

In [ ]:
gt_result[np.isnan(gt_result)] = gt_profile["nodata"]

new_profile = gt_profile.copy()
new_profile["width"], new_profile["height"] = gt_result.shape[1], gt_result.shape[0]
new_profile["crs"] = dsm_profile["crs"]
new_profile["transform"] = dsm_profile["transform"]

with rasterio.open(output_path_name, 'w', **new_profile) as dst:
    dst.write(gt_result, 1)

gt_result[gt_result==gt_profile["nodata"]] = np.nan